# ROS2 bag file converter

## Imports and variables

In [1]:
import os
from pathlib import Path

from rosbags.dataframe import get_dataframe
from rosbags.highlevel import AnyReader
from rosbags.typesys import get_types_from_msg, register_types

workspace_directory='/home/hugobaptista/ros2/AIML'

# Add all topic message types to the reader's registered types
add_types = {}
for filename in os.listdir(f'{workspace_directory}/src/ccpm_msgs/msg'):
    text = Path(f'{workspace_directory}/src/ccpm_msgs/msg/{filename}').read_text()
    name=f'ccpm_msgs/msg/{filename.split(".")[0]}'
    add_types.update(get_types_from_msg(text, name))
register_types(add_types)

## Convert from bag to Pandas dataframe

In [2]:
topics={}
dataframes={}
with AnyReader([Path(f'{workspace_directory}/AIML_bag_191223')]) as reader:
    # The 'topics' dictionary's keys are the topics names and the values are their arguments (list)
    for connection in reader.connections:
        keys = []
        for field in reader.typestore.FIELDDEFS[connection.msgtype][1]:
            keys.append(field[0])
        topics[connection.topic]=keys
    
    # The topics /motor0/status and /motor1/status are removed because they break the get_dataframe function
    topics.pop('/motor0/status')
    topics.pop('/motor1/status')

    # The 'dataframes' dictionary's keys are the topics names and the values are their pandas dataframe
    for topic in topics:
        dataframes[topic]=get_dataframe(reader, topic, topics.get(topic))

for topic in dataframes:
    print(f'Topic: {topic}\nDataframe:{dataframes[topic].head}\n-----------------------------------------\n')

Topic: /can1/receive
Dataframe:<bound method NDFrame.head of                                                                           header  \
2023-12-19 12:10:25.464663361  std_msgs__msg__Header(stamp=builtin_interfaces...   
2023-12-19 12:10:25.464672958  std_msgs__msg__Header(stamp=builtin_interfaces...   
2023-12-19 12:10:25.471074967  std_msgs__msg__Header(stamp=builtin_interfaces...   
2023-12-19 12:10:25.475840690  std_msgs__msg__Header(stamp=builtin_interfaces...   
2023-12-19 12:10:25.481377638  std_msgs__msg__Header(stamp=builtin_interfaces...   
...                                                                          ...   
2023-12-19 12:11:27.300069626  std_msgs__msg__Header(stamp=builtin_interfaces...   
2023-12-19 12:11:27.309205059  std_msgs__msg__Header(stamp=builtin_interfaces...   
2023-12-19 12:11:27.309903959  std_msgs__msg__Header(stamp=builtin_interfaces...   
2023-12-19 12:11:27.318990550  std_msgs__msg__Header(stamp=builtin_interfaces...   
2023-12-19 12:1

## Save the Pandas dataframes as csv files

In [3]:
for topic in dataframes:
    dataframes[topic].to_csv(f'{workspace_directory}/csv/{topic[1:].replace("/", "_")}.csv', index=False)
    print(f'The topic {topic} was saved in {workspace_directory}/csv/{topic[1:].replace("/", "_")}.csv')

The topic /can1/receive was saved in /home/hugobaptista/ros2/AIML/csv/can1_receive.csv
The topic /can0/receive was saved in /home/hugobaptista/ros2/AIML/csv/can0_receive.csv
The topic /device/status was saved in /home/hugobaptista/ros2/AIML/csv/device_status.csv
The topic /can0/transmit was saved in /home/hugobaptista/ros2/AIML/csv/can0_transmit.csv
The topic /modem/status was saved in /home/hugobaptista/ros2/AIML/csv/modem_status.csv
The topic /gps/receive was saved in /home/hugobaptista/ros2/AIML/csv/gps_receive.csv
The topic /can1/transmit was saved in /home/hugobaptista/ros2/AIML/csv/can1_transmit.csv
The topic /events/write_split was saved in /home/hugobaptista/ros2/AIML/csv/events_write_split.csv
The topic /parameter_events was saved in /home/hugobaptista/ros2/AIML/csv/parameter_events.csv
The topic /rosout was saved in /home/hugobaptista/ros2/AIML/csv/rosout.csv


## Errors

### /motor0/status topic

In [4]:
topics={}
dataframes={}
with AnyReader([Path(f'{workspace_directory}/AIML_bag_191223')]) as reader:
    # The 'topics' dictionary's keys are the topics names and the values are their arguments (list)
    for connection in reader.connections:
        keys = []
        for field in reader.typestore.FIELDDEFS[connection.msgtype][1]:
            keys.append(field[0])
        topics[connection.topic]=keys
    
    motor0df=get_dataframe(reader, '/motor0/status', topics.get('/motor0/status'))
    print(f'Topic: {topic}\nDataframe:{motor0df.head}\n-----------------------------------------\n')

AssertionError: 

### /motor1/status topic

In [5]:
topics={}
dataframes={}
with AnyReader([Path(f'{workspace_directory}/AIML_bag_191223')]) as reader:
    # The 'topics' dictionary's keys are the topics names and the values are their arguments (list)
    for connection in reader.connections:
        keys = []
        for field in reader.typestore.FIELDDEFS[connection.msgtype][1]:
            keys.append(field[0])
        topics[connection.topic]=keys

    motor1df=get_dataframe(reader, '/motor1/status', topics.get('/motor1/status'))
    print(f'Topic: {topic}\nDataframe:{motor1df.head}\n-----------------------------------------\n')

error: unpack_from requires a buffer of at least 77 bytes for unpacking 1 bytes at offset 76 (actual buffer size is 76)